In [25]:
# script to compute independent metrics on MRI-gad, MRI-nogad, MRI-degad, MRI-synthsr and paired metrics (MRI-degad to MRI-gad), (MRI-synthsr to MRI-gad),(MRI-degad to MRI-nogad),(MRI-synthsr to MRI-nogad)

In [22]:
from glob import glob
import os
import numpy as np
from collections import defaultdict
import monai.metrics 
import nibabel as nib
import torch
import matplotlib.pyplot as plt
import tempfile
import shutil
import pandas as pd
import math
from torchvision.transforms import CenterCrop
from monai.networks.nets import UNet
from monai.inferers import sliding_window_inference
from monai.data import CacheDataset, Dataset ,nifti_saver, PatchDataset, DataLoader, PersistentDataset
from monai.transforms import (
    Compose,
    LoadImaged,
    RandCropByPosNegLabeld,
    EnsureChannelFirstd,
    ScaleIntensityd,
    CenterSpatialCropd,
    CenterSpatialCrop,
    SpatialCrop,
    Spacingd)

import torch
from torch.nn.functional import mse_loss
from ignite.metrics import SSIM


In [27]:
date="July5"#date of model of outputs we're computing stats on
model="UNET"

In [3]:
MRI_degad= sorted(glob(f'/home/fogunsan/scratch/degad/derivatives/{model}/{date}/degad_outputs/bids/sub-*/ses-pre/anat/sub-*_acq-degad_T1w.nii.gz'))# gad images who's corresponding nongad images underwent a rigid transform
MRI_synth= sorted(glob(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/synthsr/bids/sub-*/ses-pre/anat/sub-*_ses-pre_acq-gad_run-1_desc-preproc_SynthSR_T1w.nii.gz')) # nongad images which underwent a rigid transform and underwent fcm normalization
MRI_gad=sorted(glob(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_gad/sub-*/ses-pre/anat/sub-*_ses-pre_acq-gad_run-01_T1w.nii.gz') + glob(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_56_63/bids_gad/sub-*/ses-pre/anat/sub-*_ses-pre_acq-gad_run-01_T1w.nii.gz'))
MRI_nongad=sorted(glob(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_nongad/sub-*/ses-pre/anat/sub-*_ses-pre_acq-nongad_run-01_T1w.nii.gz') + glob(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_56_63/bids_nongad/sub-*/ses-pre/anat/sub-*_ses-pre_acq-nongad_run-01_T1w.nii.gz'))

In [4]:
print(len(MRI_degad))
print(len(MRI_synth))
print(len(MRI_gad))
print(len(MRI_nongad))

3
63
63
63


In [5]:
MRI_synth=[]
MRI_gad=[]
MRI_nongad=[]
list_sub=[os.path.basename(MRI_degad[i]).split("_")[0] for i in range(len(MRI_degad))]# list of degad subjects sorted based on subject name
for sub in list_sub:
    MRI_synth.append(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/synthsr/bids/{sub}/ses-pre/anat/{sub}_ses-pre_acq-gad_run-1_desc-preproc_SynthSR_T1w.nii.gz') # nongad images which underwent a rigid transform and underwent fcm normalization
    if sub <= 'sub-P055':
        MRI_gad.append(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_gad/{sub}/ses-pre/anat/{sub}_ses-pre_acq-gad_run-01_T1w.nii.gz')
        MRI_nongad.append(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_nongad/{sub}/ses-pre/anat/{sub}_ses-pre_acq-nongad_run-01_T1w.nii.gz')
     
    else:    
        MRI_gad.append(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_56_63/bids_gad/{sub}/ses-pre/anat/{sub}_ses-pre_acq-gad_run-01_T1w.nii.gz')
        MRI_nongad.append(f'/project/6050199/akhanf/cfmm-bids/data/Lau/degad/derivatives/degad_bids_56_63/bids_nongad/{sub}/ses-pre/anat/{sub}_ses-pre_acq-nongad_run-01_T1w.nii.gz')


In [6]:
def crop_3D(img,crop_height,crop_width,crop_depth):
    
    new_top = int(round((img.shape[-3] - crop_height) / 2.0))
    new_left = int(round((img.shape[-2] - crop_width) / 2.0))
    new_front = int(round((img.shape[-1] - crop_depth) / 2.0))
    
    new_bottom = new_top + crop_height
    new_right= new_left + crop_width
    new_back = new_front+crop_depth
    return img[..., new_top:new_bottom,new_left:new_right,new_front:new_back]

In [7]:
for List in [MRI_gad,MRI_nongad,MRI_degad,MRI_synth]:
    List = sorted(List, key=lambda x: os.path.basename(x)) # sort items in list based on basename

In [8]:
# also need to make sure all images im reading in are in the space(gad space?), confirm with lab


gad_degad = [{"image1": gad_name, "image2": degad_name} for gad_name, degad_name in zip(MRI_gad,MRI_degad)] #creates list of dictionaries, with gad and nongad images labelled
gad_synth = [{"image1": gad_name, "image2": synth_name} for gad_name, synth_name in zip(MRI_gad,MRI_synth)] #creates list of dictionaries, with gad and nongad images labelled
nogad_degad = [{"image1": nogad_name, "image2": degad_name} for nogad_name, degad_name in zip(MRI_nongad,MRI_degad)] #creates list of dictionaries, with gad and nongad images labelled
nogad_synth = [{"image1": nogad_name, "image2": synth_name} for nogad_name, synth_name in zip(MRI_nongad,MRI_synth)] #creates list of dictionaries, with gad and nongad images labelled


In [9]:
def load_crop(list_dicts):
    """
    loops thru image pairs, cropped to min dimension in each direction and replaces file name in dictionary with tensor
    """
    for i in range(len(list_dicts)): # loop thru elements in list
        new_data = {}# creating new local dict to store tensors
        for image in ["image1", "image2"]:
            nifti_image = nib.load(list_dicts[i][image])
            image_data = nifti_image.get_fdata()
            new_data[image] = image_data # updating keys
            #print(new_data[image])  # Print the modified tensor inside the loop
        list_dicts[i].update(new_data)
        image1_shape, image2_shape = list_dicts[i]['image1'].shape,list_dicts[i]['image2'].shape
        min_dims = [min(dim1, dim2) for dim1, dim2 in zip(image1_shape, image2_shape)] # getting min of each dimension
        new_data['image1'],new_data['image2'] = crop_3D(new_data['image1'],*min_dims),crop_3D(new_data['image2'],*min_dims)
        new_data['image1'],new_data['image2'] = torch.from_numpy(new_data['image1']), torch.from_numpy(new_data['image2'])
        list_dicts[i].update(new_data)
    return list_dicts

In [10]:
for list_dicts in [gad_degad,gad_synth,nogad_degad,nogad_synth]:
    list_dicts = load_crop(list_dicts) # crops minimum

In [14]:
def compute_metrics(image_list,num_subs):
    """
    computing pair-wise metrics on image pairs, returning list of metrics
    """
    l1 = torch.nn.L1Loss(reduction='mean')
    mae = 0
    rmse = 0
    #ssim = 0
    for i in range(num_subs):
        mae = mae+ l1(image_list[i]["image1"],image_list[i]["image2"])
        rmse = rmse + math.sqrt(mse_loss(input=image_list[i]["image1"],target=image_list[i]["image2"]))

        # Calculate the SSIM index
        range1=float(image_list[i]["image1"].max()- image_list[i]["image1"].min())
        range2 = float(image_list[i]["image2"].max()- image_list[i]["image2"].min())
        max_range = max(range1,range2) 
        #replace this ssim with a 3d implementation or compute each 2d slice and take avg
        """
        ssim_loss = SSIM(data_range=max_range)
        
        ssim_loss.update((image_list[i]["image1"], image_list[i]["image2"]))
        ssim+= ssim_loss.compute()
        """ 
        ##TODO: ADD OTHER METRICS
    mae /= num_subs
    rmse /= num_subs
    #ssim/= num_subs
    return [mae,rmse]

In [19]:
num_subs = len(gad_degad)
image_metrics_list = [0] * 4  # initalize list of list which will hold list of metrics for each dict

for i, image_list in enumerate([gad_degad, gad_synth, nogad_degad, nogad_synth]):
    metrics = compute_metrics(image_list, num_subs)
    image_metrics_list[i] = metrics

image_metrics_list = np.array(image_metrics_list)

In [24]:
image_list_metrics = np.transpose(image_metrics_list)

In [25]:
column_names = ['gad_degad', 'gad_synth', 'nogad_degad', 'nogad_synth']
row_index_labels = ['MAE', 'RMSE']
df = pd.DataFrame(image_list_metrics , columns=column_names, index=row_index_labels)

In [28]:
print(df)

       gad_degad   gad_synth  nogad_degad  nogad_synth
MAE   411.125238  371.786535   264.127308   229.963985
RMSE  843.043774  797.289239   522.728891   480.714862


In [29]:
output_dir=f'/home/fogunsan/scratch/degad/derivatives/{model}/{date}/metrics'

In [30]:
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [31]:
df.to_csv(f'{output_dir}/metrics.csv', index=False)